In [1]:
import pandas as pd
import numpy as np

In [2]:
xls_IT_1 = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/preview_all.xlsx')
df_IT_1 = pd.read_excel(xls_IT_1)

In [3]:
#Benchmark trading data
xls_index = pd.ExcelFile(r'/Users/hongnhungvu/Desktop/Thesis_Data/Data_IT/Data_Index_2011_2021.xlsx')
df_index = pd.read_excel(xls_index)
df_index.rename(columns={"TradingDate":"RegisterDate","IndexCode":"Exchange"}, inplace=True)

df_index['Exchange'] = df_index['Exchange'].replace(['VNINDEX','HNXINDEX','UPCOMINDEX'],['VNI','HNX','UPCOM'])

df_VNI = df_index[df_index['Exchange']=='VNI'].copy()
df_HNX = df_index[df_index['Exchange']=='HNX'].copy()
df_UPCOM = df_index[df_index['Exchange']=='UPCOM'].copy()

#Calculate daily ret of benchmarks
df_VNI['Rm_ret'] = df_VNI['CloseIndex'].pct_change(-1)
df_HNX['Rm_ret'] = df_HNX['CloseIndex'].pct_change(-1)
df_UPCOM['Rm_ret'] = df_UPCOM['CloseIndex'].pct_change(-1)

In [4]:
#Match Benchmark Ret UPCOM
df_IT_UPCOM = df_IT_1[df_IT_1['Exchange']=='UPCOM'].copy()

for x in range(-250,0):
    df_IT_UPCOM['Rm_T1_' + str(x)] = df_IT_UPCOM.set_index('RegisterDate').index.map(df_UPCOM.set_index('RegisterDate')['Rm_ret'].shift(x))

df_IT_UPCOM['Rm_T1_0'] = df_IT_UPCOM.set_index('RegisterDate').index.map(df_UPCOM.set_index('RegisterDate')['Rm_ret'])

for x in range(1,10):
    df_IT_UPCOM['Rm_T1_' + str(x)] = df_IT_UPCOM.set_index('RegisterDate').index.map(df_UPCOM.set_index('RegisterDate')['Rm_ret'].shift(x))

In [5]:
import statsmodels.api as sm

#Regress Ri on Rm

def regress(data):
    result_all=[]
    for i in range(len(data)):
        Y = list(data.iloc[i][32:253])
        X = list(data.iloc[i][292:513])
        X = sm.add_constant(X)
        result = sm.OLS(Y,X).fit()
        result_all.append(result.params)
    return result_all  

#Why slice [32:253] and [292:513]
#df_IT_UPCOM.columns.get_indexer(['Ri_T1_-250', 'Ri_T1_-30','Rm_T1_-250', 'Rm_T1_-30'])
#array([ 32, 252, 292, 512])

#Run for HNX
res_UPCOM = regress(df_IT_UPCOM)

In [6]:
#Insert alphas & betas into df_IT_UPCOM
df_IT_UPCOM['alpha'] = np.array(res_UPCOM)[:,0]
df_IT_UPCOM['beta'] = np.array(res_UPCOM)[:,1]

df_IT_UPCOM = df_IT_UPCOM.drop(df_IT_UPCOM[(df_IT_UPCOM.alpha == 0) & (df_IT_UPCOM.beta == 0)].index)
df_IT_UPCOM = df_IT_UPCOM.drop(df_IT_UPCOM[(df_IT_UPCOM.alpha.isnull()) & (df_IT_UPCOM.beta.isnull())].index)

In [7]:
#Calculate sd of abnormal return of each stock in each event in period (-250,-30)

def sd_ar(data):
    sd_ar_all=[]
    for i in range(len(data)):
        R_i = list(data.iloc[i][32:253])
        R_m = list(data.iloc[i][292:513])
        ar = [s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(R_i,R_m)]
        sd_ar_all.append([np.nanstd(ar)])
    return sd_ar_all

sd_UPCOM = sd_ar(df_IT_UPCOM)

In [8]:
#Insert sd of abnormal return into df_IT_VNI
df_IT_UPCOM['sd_ar'] = np.array(sd_UPCOM)[:,0]

In [9]:
#Calculate cumulative abnormal return (-30,-1)

def cal_car_before(data):
    car_all=[]
    for i in range(len(data)):
        r_i = list(data.iloc[i][252:282])
        r_m = list(data.iloc[i][512:542])
        car = np.nansum([s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(r_i,r_m)])
        car_all.append(car)
    return car_all

#Why slice [252:282] and [512:542]
#df_IT_UPCOM.columns.get_indexer(['Ri_T1_-30', 'Ri_T1_-1','Rm_T1_-30', 'Rm_T1_-1'])
#array([252, 281, 512, 541])

cal_before_UPCOM = cal_car_before(df_IT_UPCOM)

In [10]:
#Insert car before into df_IT_HNX
df_IT_UPCOM['car_before'] = np.array(cal_before_UPCOM)

In [11]:
df_IT_UPCOM['ar_T0'] = df_IT_UPCOM['Ri_T1_0'] - (df_IT_UPCOM['Rm_T1_0']*df_IT_UPCOM['beta']+df_IT_UPCOM['alpha'])

def cal_car_after(data):
    car_all=[]
    for i in range(len(data)):
        r_i = list(data.iloc[i][283:292])
        r_m = list(data.iloc[i][543:552])
        car = np.nansum([s - data.iloc[i]['alpha'] - m*data.iloc[i]['beta'] for s,m in zip(r_i,r_m)])
        car_all.append(car)
    return car_all

#Why slice [283:292] and [543:552]
#df_IT_UPCOM.columns.get_indexer(['Ri_T1_1', 'Ri_T1_9','Rm_T1_1', 'Rm_T1_9'])
#array([283, 291, 543, 551])

cal_after = cal_car_after(df_IT_UPCOM)

In [12]:
#Insert car after into df_IT_UPCOM
df_IT_UPCOM['car_after'] = np.array(cal_after)

df_IT_UPCOM['car_t_t+n'] = df_IT_UPCOM['car_after'] + df_IT_UPCOM['ar_T0']

In [13]:
#Calculate t-statistics
df_IT_UPCOM['t-stat_before'] = df_IT_UPCOM['car_before']/(df_IT_UPCOM['sd_ar']*(30**0.5))
df_IT_UPCOM['t-stat_eventdate'] = df_IT_UPCOM['ar_T0']/df_IT_UPCOM['sd_ar']
df_IT_UPCOM['t-stat_after'] = df_IT_UPCOM['car_after']/(df_IT_UPCOM['sd_ar']*((df_IT_UPCOM['Diff_days']-1)**0.5))
df_IT_UPCOM['t-stat_t_t+n'] = df_IT_UPCOM['car_t_t+n']/(df_IT_UPCOM['sd_ar']*(df_IT_UPCOM['Diff_days']**0.5))

df_IT_UPCOM

,StockCode,RegisterDate,ResultReportDate,DateStartExpected,DateEndExpected,DateActionFrom,DateActionTo,TypeName,RegisterVolumeBefore,RegisterVolumeBeforePercent,...,beta,sd_ar,car_before,ar_T0,car_after,car_t_t+n,t-stat_before,t-stat_eventdate,t-stat_after,t-stat_t_t+n
219,GTH,2012-07-20,2012-08-28,2012-07-23,2012-08-21,2012-07-23,2012-08-22,GD CD noi bo,416409,0.173504,...,1.960419e+00,6.841502e-02,3.063789e-01,1.019924e-01,0.000000e+00,1.019924e-01,0.817611,1.490790,NaN,1.490790
244,HFX,2012-08-02,2012-09-18,2012-08-06,2012-09-05,2012-08-06,2012-09-05,GD CD noi bo,41260,0.032488,...,-2.279236e+00,1.029748e-01,-1.344608e-01,-1.180778e-02,-7.569256e-04,-1.256471e-02,-0.238399,-0.114667,-0.007351,-0.086279
277,CI5,2012-09-19,2012-10-05,2012-09-24,2012-10-19,2012-09-28,2012-09-28,GD CD noi bo,22800,0.018240,...,8.026156e-07,7.165680e-08,-1.718336e-07,-2.145603e-09,-8.351022e-09,-1.049662e-08,-0.437814,-0.029943,-0.082408,-0.084573
353,CAD,2012-11-08,2012-11-28,2012-11-12,2012-12-11,2012-11-09,2012-11-23,GD CD noi bo,1000000,0.048077,...,-3.240047e-02,5.738870e-02,7.751763e-01,4.690183e-03,-6.198443e-02,-5.729424e-02,2.466116,0.081727,-1.080081,-0.705943
461,SHP,2012-12-14,2013-02-20,2012-12-18,2013-01-16,2012-12-19,2013-01-10,GD CD noi bo,73250,0.000782,...,4.006730e-01,3.393257e-02,-2.771597e-01,7.100425e-03,6.483951e-03,1.358438e-02,-1.491258,0.209251,0.191083,0.283079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11899,SID,2020-11-13,2020-12-22,2020-11-17,2020-12-15,2020-11-16,2020-12-15,GD cua nguoi lien quan,96092796,0.960928,...,3.198349e-01,5.048799e-02,1.313522e-01,-1.016289e-02,9.465995e-02,8.449706e-02,0.474995,-0.201293,1.874900,1.183419
11903,PVP,2020-11-18,2020-12-23,2020-11-23,2020-12-22,2020-11-23,2020-12-22,GD cua nguoi lien quan,9543747,0.101233,...,1.300244e+00,3.057229e-02,6.182291e-02,6.527914e-02,2.074701e-01,2.727492e-01,0.369199,2.135239,4.798578,5.150803
11918,VRG,2020-12-04,2020-12-25,2020-12-07,2021-01-05,2020-12-07,2020-12-23,GD cua nguoi lien quan,1414000,0.054605,...,1.792752e+00,3.672334e-02,-2.240888e-01,9.197547e-03,0.000000e+00,9.197547e-03,-1.114083,0.250455,NaN,0.250455
11920,CC4,2020-11-13,2020-12-28,2020-11-17,2020-12-15,2020-11-16,2020-12-15,GD cua nguoi lien quan,2880000,0.180000,...,5.574450e-01,5.402958e-02,3.173864e-01,1.209345e-01,-2.833739e-03,1.181007e-01,1.072497,2.238301,-0.052448,1.545632


In [14]:
df_IT_UPCOM['StockCode'].nunique()

422

In [15]:
df_UPCOM_buy = pd.DataFrame(df_IT_UPCOM[(df_IT_UPCOM['RegisterSellVolume'].isnull())])
df_UPCOM_sell = pd.DataFrame(df_IT_UPCOM[(df_IT_UPCOM['RegisterBuyVolume'].isnull())])

In [16]:
#Form groups of events for t-test & Wilcoxon test

import scipy
from scipy.stats import norm
from scipy.stats import wilcoxon

df_buy_1 = df_UPCOM_buy[(df_UPCOM_buy['Diff_days']==1)]
df_sell_1 = df_UPCOM_sell[(df_UPCOM_sell['Diff_days']==1)]

df_buy_2 = df_UPCOM_buy[(df_UPCOM_buy['Diff_days']==2)]
df_sell_2 = df_UPCOM_sell[(df_UPCOM_sell['Diff_days']==2)]

df_buy_3 = df_UPCOM_buy[(df_UPCOM_buy['Diff_days']==3)]
df_sell_3 = df_UPCOM_sell[(df_UPCOM_sell['Diff_days']==3)]

In [17]:
import scipy.integrate 

from scipy import stats

tstat_UPCOM_1_buy, pval_UPCOM_1_buy = scipy.stats.ttest_1samp(df_buy_1['car_t_t+n'],0,alternative='two-sided')
tstat_UPCOM_1_buy, pval_UPCOM_1_buy, df_buy_1['car_t_t+n'].mean(), df_buy_1['car_t_t+n'].std(), len(df_buy_1)

(0.23197375848086738,
 0.81696021544276,
 0.0012271351153079777,
 0.05770680177930159,
 119)

In [18]:
tstat_UPCOM_1_sell, pval_UPCOM_1_sell = scipy.stats.ttest_1samp(df_sell_1['car_t_t+n'],0,alternative='two-sided')
tstat_UPCOM_1_sell, pval_UPCOM_1_sell, df_sell_1['car_t_t+n'].mean(), df_sell_1['car_t_t+n'].std(), len(df_sell_1)

(-3.1531599748710613,
 0.002096283074487177,
 -0.02103722373922911,
 0.06933528395056253,
 108)

In [19]:
tstat_UPCOM_2_buy, pval_UPCOM_2_buy = scipy.stats.ttest_1samp(df_buy_2['car_t_t+n'],0,alternative='two-sided')
tstat_UPCOM_2_buy, pval_UPCOM_2_buy, df_buy_2['car_t_t+n'].mean(), df_buy_2['car_t_t+n'].std(), len(df_buy_2)

(1.8280453571836592,
 0.06832047976796808,
 0.008591582776386301,
 0.09209834450695864,
 384)

In [20]:
tstat_UPCOM_2_sell, pval_UPCOM_2_sell = scipy.stats.ttest_1samp(df_sell_2['car_t_t+n'],0,alternative='two-sided')
tstat_UPCOM_2_sell, pval_UPCOM_2_sell, df_sell_2['car_t_t+n'].mean(), df_sell_2['car_t_t+n'].std(), len(df_sell_2)

(-1.11133018795183,
 0.2672710839695883,
 -0.0044412868864695575,
 0.07115334916392192,
 317)

In [21]:
tstat_UPCOM_3_buy, pval_UPCOM_3_buy = scipy.stats.ttest_1samp(df_buy_3['car_t_t+n'],0,alternative='two-sided')
tstat_UPCOM_3_buy, pval_UPCOM_3_buy, df_buy_3['car_t_t+n'].mean(), df_buy_3['car_t_t+n'].std(), len(df_buy_3)

(2.7257990757364836,
 0.006953571734189683,
 0.018216994025485142,
 0.09730850636675448,
 212)

In [22]:
tstat_UPCOM_3_sell, pval_UPCOM_3_sell = scipy.stats.ttest_1samp(df_sell_3['car_t_t+n'],0,alternative='two-sided')
tstat_UPCOM_3_sell, pval_UPCOM_3_sell, df_sell_3['car_t_t+n'].mean(), df_sell_3['car_t_t+n'].std(), len(df_sell_3)

(-0.33140203557035364,
 0.7407263401716377,
 -0.0020227036870820946,
 0.08211384374208323,
 181)

In [23]:
import scipy
from scipy.stats import norm
from scipy.stats import wilcoxon

#The wilcoxon function below does not return z-score 
#so I have to reverse-engineer z-score from p-value: norm.ppf(p-value/2) as we conduct a two-sided test

w1b, p1b = wilcoxon(df_buy_1["car_t_t+n"])
w1b, p1b, norm.ppf(p1b/2), df_buy_1["car_t_t+n"].median()

(3406.0, 0.6636335827110497, -0.43490197260639446, -0.0005787732179168477)

In [24]:
w1s, p1s = wilcoxon(df_sell_1["car_t_t+n"])
w1s, p1s, norm.ppf(p1s/2), df_sell_1["car_t_t+n"].median()

(2147.0, 0.01469194990140387, -2.439884709530312, -0.0023648242642638688)

In [25]:
w2b, p2b = wilcoxon(df_buy_2["car_t_t+n"])
w2b, p2b, norm.ppf(p2b/2), df_buy_2["car_t_t+n"].median()

(34609.0, 0.28005781605967395, -1.0801894709734952, -0.001376357349928349)

In [26]:
w2s, p2s = wilcoxon(df_sell_2["car_t_t+n"])
w2s, p2s, norm.ppf(p2s/2), df_sell_2["car_t_t+n"].median()

(19020.0, 0.00015368404821974988, -3.785039691281219, -0.004049508400300095)

In [27]:
w3b, p3b = wilcoxon(df_buy_3["car_t_t+n"])
w3b, p3b, norm.ppf(p3b/2), df_buy_3["car_t_t+n"].median()

(10393.0, 0.3163460608557669, -1.0019948899846196, -3.1079484817228855e-05)

In [28]:
w3s, p3s = wilcoxon(df_sell_3["car_t_t+n"])
w3s, p3s, norm.ppf(p3s/2), df_sell_3["car_t_t+n"].median()

(7395.0, 0.23375243454166184, -1.1907482385388888, -0.004240348168774829)